In [8]:
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from IPython.display import clear_output
import time
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
import json
import vlc

In [32]:
#Open ChromeDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get("https://www.amazon.in/s?k=Dune&i=stripbooks")



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\harik\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-32-a9184be2b85f>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Low and High Resolution Book Image

In [4]:
def download_image(book_name, author="", num_images=5):
    
    query = book_name + " " + author
    page_url = f"https://www.amazon.in/s?k={query.strip()}&i=stripbooks"
    print("Searching for:", query)
    driver.get(page_url)
    print("Done!")
    items = driver.find_elements(By.CLASS_NAME, "s-result-item")
    images = driver.find_elements(By.CLASS_NAME, "s-image")


    i=0
    match = 0
    while True:
        if "Sponsored" in items[i].text or not book_name.lower() in items[i].text.lower():
            i+=1
            pass
        else:
            match+=1
            link1 = images[i-1].get_attribute("src")
            print("link1:", link1)
            images[i-1].click()
            driver.switch_to.window(driver.window_handles[-1])
            # time.sleep(1)
            try:
                driver.find_elements(By.CLASS_NAME, "thumb-text")[0].click()
                link2 = driver.find_elements(By.ID, "igImage")[0].get_attribute("src")
                print("link2:", link2)
                print("Closing the Extra Window")
                driver.close()
                print("Switching to the Main Window")
                driver.switch_to.window(driver.window_handles[0])
            except:
                print("Closing the Extra Window")
                driver.close()
                print("Switching to the Main Window")
                driver.switch_to.window(driver.window_handles[0])
                
            #Save images using link
            response = requests.get(link1)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            print(f"Image shape is {w}x{h}")
            img_path = os.path.join(os.getcwd(), "img")
            file_name_low = f"{book_name}_{match}_{w}x{h}.jpg"
            img.save(os.path.join(img_path, "low", file_name_low))

            #Higher resolution
            response = requests.get(link2)
            img2 = Image.open(BytesIO(response.content))
            w, h = img2.size
            print(f"Image shape of higher resultion is {w}x{h}")
            file_name_high = f"h_{book_name}_{match}_{w}x{h}.jpg"
            img2.save(os.path.join(img_path, "high", file_name_high))
            i+=1
            clear_output()
        if match==num_images or i==10:
            break
    

In [5]:
book_dict= json.load(open("book_dict.json"))

In [10]:
p.stop()

In [7]:
download_image("Rhythm of War", "Sanderson", num_images=3)


In [84]:
book_dict = {}

In [85]:
book_dict['items'] = []

In [86]:
json.dump(book_dict, open("book_dict_links.json", "w"))

In [82]:
book_dict['items'].append({"Emma":{
    "author": "Jane Austen",
}})

In [83]:
book_dict

{'items': [{'Emma': {'author': 'Jane Austen'}}]}

In [41]:
def download_image(book_name, author="", num_images=3):
    
    query = book_name + " " + author.split(" ")[-1]
    page_url = f"https://www.amazon.in/s?k={query.strip()}&i=stripbooks"
    print("Searching for:", query)
    driver.get(page_url)
    print("Done!")
    items = driver.find_elements(By.CLASS_NAME, "s-result-item")
    images = driver.find_elements(By.CLASS_NAME, "s-image")

    low_links = []
    high_links = []
    low_file_paths = []
    high_file_paths = []
    i=0
    match = 0
    while True:
        if "Sponsored" in items[i].text or not book_name.lower() in items[i].text.lower():
            i+=1
            pass
        else:
            match+=1
            link1 = images[i-1].get_attribute("src")
            low_links.append(link1)
            print("link1:", link1)
            images[i-1].click()
            driver.switch_to.window(driver.window_handles[-1])
            # time.sleep(1)
            link2 = ""
            try:
                driver.find_elements(By.CLASS_NAME, "thumb-text")[0].click()
                link2 = driver.find_elements(By.ID, "igImage")[0].get_attribute("src")
                high_links.append(link2)
                print("link2:", link2)
                print("Closing the Extra Window")
                driver.close()
                print("Switching to the Main Window")
                driver.switch_to.window(driver.window_handles[0])
            except:
                high_links.append(None)
                print("Closing the Extra Window")
                driver.close()
                print("Switching to the Main Window")
                driver.switch_to.window(driver.window_handles[0])

            if link1:   
                #Save images using link
                response = requests.get(link1)
                img = Image.open(BytesIO(response.content))
                w, h = img.size
                print(f"Image shape is {w}x{h}")
                img_path = os.path.join(os.getcwd(), "img")
                file_name_low = f"{book_name}_{match}_{w}x{h}.jpg"
                file_path_low = os.path.join(img_path, "low", file_name_low)
                low_file_paths.append(file_path_low)
                img.save(file_path_low)

            if link2:
                #Higher resolution
                response = requests.get(link2)
                img2 = Image.open(BytesIO(response.content))
                w, h = img2.size
                print(f"Image shape of higher resultion is {w}x{h}")
                file_name_high = f"h_{book_name}_{match}_{w}x{h}.jpg"
                file_path_high = os.path.join(img_path, "high", file_name_high)
                high_file_paths.append(file_path_high)
                img2.save(file_path_high)
            i+=1
            clear_output()
        if match==num_images or i==10:
            break
    book_dict = json.load(open("book_dict_links.json"))
    book_dict['items'].append({
    "name":book_name,
    "author": author,
    "low_links": low_links,
    "high_links": high_links,
    "low_file_paths": low_file_paths,
    "high_file_paths": high_file_paths,
                    })
    json.dump(book_dict, open("book_dict_links.json", "w"))


In [43]:
for i in range(39,40):
    name = book_dict['items'][i]['name'].split('(')[0].strip()
    print(name)

Har ry Potter and the Philosopher's Stone


In [44]:
df = pd.read_csv("books_info.csv")

In [56]:
book = df[df['Author']=="Brandon Sanderson"]['Title']
book.iloc[0].split('(')[0].strip()

'Cytonic'

In [57]:
book = df[df['Author']=="Brandon Sanderson"]['Title']
author = 'Brandon Sanderson'
print("Total:" + str(len(book)))
try:
    for i in range(len(book)):
        name = book.iloc[i].split('(')[0].strip()
        download_image(name, author, num_images=3)
        print(f"{str(i+1)} Done!\n{str(len(book)-i-1)} left")
except Exception as e:
    print(i)
    p = vlc.MediaPlayer(r'C:\Users\harik\Music\My Media\notification\the_tardis.mp3')
    p.play()
    print(e.with_traceback())

47 Done!
0 left


In [40]:
link2

NameError: name 'link2' is not defined

In [9]:
books = ['Dune', 'Pride and Prejudice', 'The Eye of the World', 'The Name of the Wind', 'Harry Potter and the Prisoner of Azkaban']
authors = ['Frank Herbert', 'Jane Austen', 'Robert Jordan', 'Patrick Rothfuss', 'J.K. Rowling']
map(download_image, books, authors)
for book, author in list(zip(books, authors)):
    download_image(book, author, num_images=3)

In [31]:
try:
    1/0
except Exception as e:
    print(e.with_traceback())

TypeError: with_traceback() takes exactly one argument (0 given)

In [27]:
items = driver.find_elements(By.CLASS_NAME, "s-result-item")
images = driver.find_elements(By.CLASS_NAME, "s-image")
book_name = "Circe"
low_links = []
high_links = []
low_file_paths = []
high_file_paths = []
i=0
match = 0
while True:
    if "Sponsored" in items[i].text or not book_name.lower() in items[i].text.lower():
        i+=1
        pass
    else:
        match+=1
        link1 = images[i-1].get_attribute("src")
        low_links.append(link1)
        print("link1:", link1)
        images[i-1].click()
        driver.switch_to.window(driver.window_handles[-1])
        # time.sleep(1)
        try:
            driver.find_elements(By.CLASS_NAME, "thumb-text")[0].click()
            link2 = driver.find_elements(By.ID, "igImage")[0].get_attribute("src")
            high_links.append(link2)
            print("link2:", link2)
            print("Closing the Extra Window")
            driver.close()
            print("Switching to the Main Window")
            driver.switch_to.window(driver.window_handles[0])
        except:
            high_links.append(None)
            print("Closing the Extra Window")
            driver.close()
            print("Switching to the Main Window")
            driver.switch_to.window(driver.window_handles[0])

link1: https://m.media-amazon.com/images/I/41uH4+3DNjL._AC_UY218_.jpg
Closing the Extra Window
Switching to the Main Window
link1: https://m.media-amazon.com/images/I/41uH4+3DNjL._AC_UY218_.jpg
Closing the Extra Window
Switching to the Main Window
link1: https://m.media-amazon.com/images/I/41uH4+3DNjL._AC_UY218_.jpg
Closing the Extra Window
Switching to the Main Window
link1: https://m.media-amazon.com/images/I/41uH4+3DNjL._AC_UY218_.jpg
Closing the Extra Window
Switching to the Main Window
link1: https://m.media-amazon.com/images/I/41uH4+3DNjL._AC_UY218_.jpg
Closing the Extra Window
Switching to the Main Window
link1: https://m.media-amazon.com/images/I/41uH4+3DNjL._AC_UY218_.jpg
Closing the Extra Window
Switching to the Main Window
link1: https://m.media-amazon.com/images/I/41uH4+3DNjL._AC_UY218_.jpg
Closing the Extra Window
Switching to the Main Window
link1: https://m.media-amazon.com/images/I/41uH4+3DNjL._AC_UY218_.jpg
Closing the Extra Window
Switching to the Main Window
link1: h

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=96.0.4664.45)
Stacktrace:
Backtrace:
	Ordinal0 [0x005C6903+2517251]
	Ordinal0 [0x0055F8E1+2095329]
	Ordinal0 [0x00462848+1058888]
	Ordinal0 [0x0044BF90+966544]
	Ordinal0 [0x004A8999+1345945]
	Ordinal0 [0x004AC1D6+1360342]
	Ordinal0 [0x004A64A7+1336487]
	Ordinal0 [0x004827A7+1189799]
	Ordinal0 [0x00483609+1193481]
	GetHandleVerifier [0x00755904+1577972]
	GetHandleVerifier [0x00800B97+2279047]
	GetHandleVerifier [0x00656D09+534521]
	GetHandleVerifier [0x00655DB9+530601]
	Ordinal0 [0x00564FF9+2117625]
	Ordinal0 [0x005698A8+2136232]
	Ordinal0 [0x005699E2+2136546]
	Ordinal0 [0x00573541+2176321]
	BaseThreadInitThunk [0x75E66739+25]
	RtlGetFullPathName_UEx [0x77CF8AFF+1215]
	RtlGetFullPathName_UEx [0x77CF8ACD+1165]


In [10]:
book_dict = json.load(open("book_dict_links.json"))

In [11]:
book_dict['items'][0]['low_file_paths'][0]

'c:\\Users\\harik\\Desktop\\hari31416.github.io\\Projects\\Book_Library\\Python\\img\\low\\Dune_1_122x218.jpg'

In [12]:
os.path.relpath(book_dict['items'][0]['low_file_paths'][0], os.getcwd())

'img\\low\\Dune_1_122x218.jpg'

In [ ]:
df = pd.read_csv("books_info.csv")

In [65]:
def convert(value):
    if value is None:
        pass
    else:
        return int(value)

In [68]:
df["Number of Pages"].isnull().sum()

6

In [24]:
df[df['Title']=="Python 3 Object Oriented Programming"]['Author'].values[0]

'Dusty Phillips'

In [69]:
book_dict = {'items':[]}
for name in df['Title']:
    try:
        book_dict['items'].append({
            'name': name,
            'author': df[df['Title']==name]['Author'].values[0],
            'isbn13': int(df[df['Title']==name]['ISBN13'].values[0]),
            'publisher': df[df['Title']==name]['Publisher'].values[0],
            'avg-rating': df[df['Title']==name]['Average Rating'].values[0],
            'num-pages': int(df[df['Title']==name]['Number of Pages'].values[0]),
            'publication-year': int(df[df['Title']==name]['Original Publication Year'].values[0]),
        })
    except ValueError:
        pass


In [74]:
len(book_dict['items'])

316

In [72]:
json.dump(book_dict, open("book_dict.json", "w"))

In [ ]:
name = " ="1849511268""

In [22]:
df["ISBN"] = df['ISBN'].apply(lambda x: x.replace("\"", "").replace("=", ""))

In [25]:
df["ISBN13"]=df['ISBN13'].apply(lambda x: x.replace("\"", "").replace("=", ""))

In [ ]:
df.to

In [34]:
df.columns

Index(['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors',
       'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
       'Number of Pages', 'Year Published', 'Original Publication Year',
       'Date Read', 'Date Added', 'Bookshelves', 'Bookshelves with positions',
       'Exclusive Shelf', 'My Review', 'Spoiler', 'Private Notes',
       'Read Count', 'Recommended For', 'Recommended By', 'Owned Copies',
       'Original Purchase Date', 'Original Purchase Location', 'Condition',
       'Condition Description', 'BCID'],
      dtype='object')

In [40]:
df[['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors',
       'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
       'Number of Pages', 'Year Published', 'Original Publication Year']].to_csv("books_info.csv", index=False)